In [2]:
import ee
from datetime import date
import folium
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
from geemap import cartoee
import numpy as np
import cartopy as ccrs
import geemap.colormaps as cm


In [3]:
ee.Initialize()
import sys
sys.path.append('C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\ET_Code\\geeSEBAL-master\\etbrasil\\')
import geesebal
from geesebal import (tools,landsatcollection,masks,meteorology,endmembers, 
evapotranspiration,collection,timeseries,image)

In [17]:
ls=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate("2021-01-01","2021-12-31").filterMetadata('WRS_PATH', 'equals', 38).filterMetadata('WRS_ROW', 'equals',37).filterMetadata('CLOUD_COVER', 'less_than', 10);
ls = ls.filter(ee.Filter.neq('system:index','LC08_038037_20210903'));
ls_list=ls.aggregate_array('system:index').getInfo()
print(ls_list)
count = ls.size().getInfo()
print("Number of scenes: ", count)
n=0
k=0
time_start_list=[]
#====== ITERATIVE PROCESS ======#
#FOR EACH IMAGE ON THE LIST
#ESTIMATE ET DAILY IMAGE
while n < count:
    #GET IMAGE
    image= ls.filterMetadata('system:index','equals',ls_list[n]).first()
    image.getInfo()
    image=ee.Image(image)
        # et=image.Image(image)
    NDVI_cold=5
    Ts_cold=20
    NDVI_hot=10
    Ts_hot=20
    index=image.get('system:index')
    cloud_cover=image.get('CLOUD_COVER')
    LANDSAT_ID=image.get('LANDSAT_ID').getInfo()
    print(LANDSAT_ID)
    landsat_version=image.get('SATELLITE').getInfo()
    azimuth_angle=image.get('SOLAR_ZENITH_ANGLE')
    time_start=image.get('system:time_start')
    date=ee.Date(time_start)
    year=ee.Number(date.get('year'))
    month=ee.Number(date.get('month'))
    day=ee.Number(date.get('day'))
    hour=ee.Number(date.get('hour'))
    minuts = ee.Number(date.get('minutes'))
    crs = image.projection().crs()
    transform=ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))
    date_string=date.format('YYYY-MM-dd').getInfo()
    #ENDMEMBERS
    p_top_NDVI=ee.Number(NDVI_cold)
    p_coldest_Ts=ee.Number(Ts_cold)
    p_lowest_NDVI=ee.Number(NDVI_hot)
    p_hottest_Ts=ee.Number(Ts_hot)
    ls_trial=image.select([0,1,2,3,4,5,6,7,10], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","BRT","pixel_qa"])
    ls.first_toa=ee.Image('LANDSAT/LC08/C01/T1/'+index.getInfo())

    col_rad = ee.Algorithms.Landsat.calibratedRadiance(ls.first_toa)
    col_rad = ls_trial.addBands(col_rad.select([9],["T_RAD"]))
    #CLOUD REMOVAL
    # ls_trial=ee.ImageCollection(col_rad).map(masks.f_cloudMaskL8_SR)
    ls_trial=masks.f_cloudMaskL8_SR(ls_trial)
    #ALBEDO TASUMI ET AL. (2008) METHOD WITH KE ET AL. (2016) COEFFICIENTS
    # ls_trial=ls_trial.map(masks.f_albedoL8)
    ls_trial=masks.f_albedoL8(ls_trial)
    #------ Meteorology
      #GEOMETRY
    geometryReducer=ls_trial.geometry().bounds().getInfo()
    geometry_download=geometryReducer['coordinates']
    # camada_clip=ls_trial.select('BRT').first()
    camada_clip=ls_trial.select('BRT')
    sun_elevation=ee.Number(90).subtract(ee.Number(azimuth_angle))
    #METEOROLOGY PARAMETERS
    col_meteorology= meteorology.get_meteorology(ls_trial,time_start);

    #AIR TEMPERATURE [C]
    T_air = col_meteorology.select('AirT_G');

    #WIND SPEED [M S-1]
    ux= col_meteorology.select('ux_G');
    #RELATIVE HUMIDITY [%]
    UR = col_meteorology.select('RH_G');

    #NET RADIATION 24H [W M-2]
    Rn24hobs = col_meteorology.select('Rn24h_G');
    #------
    #------ Elevation
    #SRTM DATA ELEVATION
    SRTM_ELEVATION ='USGS/SRTMGL1_003'
    srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
    z_alt = srtm.select('elevation')

#     GET IMAGE
#     ls_trial=ls_trial.first()

#     SPECTRAL IMAGES (NDVI, EVI, SAVI, LAI, T_LST, e_0, e_NB, long, lat)
    ls_trial=tools.fexp_spec_ind(ls_trial)
    ls_trial=tools.LST_DEM_correction(ls_trial, z_alt, T_air, UR,sun_elevation,hour,minuts)
    #COLD PIXEL
    d_cold_pixel=endmembers.fexp_cold_pixel(ls_trial, geometryReducer, p_top_NDVI, p_coldest_Ts)
    #COLD PIXEL NUMBER
    n_Ts_cold = ee.Number(d_cold_pixel.get('temp').getInfo())
    #INSTANTANEOUS OUTGOING LONG-WAVE RADIATION [W M-2]
    ls_trial=tools.fexp_radlong_up(ls_trial)
    #INSTANTANEOUS INCOMING SHORT-WAVE RADIATION [W M-2]
    ls_trial=tools.fexp_radshort_down(ls_trial,z_alt,T_air,UR, sun_elevation)
    #INSTANTANEOUS INCOMING LONGWAVE RADIATION [W M-2]
    ls_trial=tools.fexp_radlong_down(ls_trial, n_Ts_cold)
    #INSTANTANEOUS NET RADIATON BALANCE [W M-2]
#     print(ls_trial.select('Rs_down').getInfo())
#     print(ls_trial.select('Rl_down').getInfo())
#     print(ls_trial.select('Rl_up').getInfo())

    ls_trial=tools.fexp_radbalance(ls_trial)
#     print(ls_trial.select('Rn').getInfo())

    #SOIL HEAT FLUX (G) [W M-2]
    ls_trial=tools.fexp_soil_heat(ls_trial)
    #HOT PIXEL
    d_hot_pixel=endmembers.fexp_hot_pixel(ls_trial, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
    #SENSIBLE HEAT FLUX (H) [W M-2]
    ls_trial=tools.fexp_sensible_heat_flux(ls_trial, ux, UR,Rn24hobs,n_Ts_cold,
                                   d_hot_pixel, date_string,geometryReducer)

    #DAILY EVAPOTRANSPIRATION (ET_24H) [MM DAY-1]
    ls_trial=evapotranspiration.fexp_et(ls_trial,Rn24hobs)

    NAME_FINAL=LANDSAT_ID[:5]+LANDSAT_ID[10:17]+LANDSAT_ID[17:25]
    ls_trial=ls_trial.addBands([ls_trial.select('ET_24h').rename(NAME_FINAL)])
    ET_daily=ls_trial.select(['ET_24h'],[NAME_FINAL])
    time_start_list.append(str(hour.getInfo())+str(minuts.getInfo()))
    if k ==0:
        Collection_ET=ET_daily
        Collection_H=ls_trial.select(['H'],[NAME_FINAL])
        Collection_Rn_daily=Rn24hobs.select(['Rn24h_G'],[NAME_FINAL])
                  #AIR TEMPERATURE [C]
        Collection_Ta=T_air.select(['AirT_G'],[NAME_FINAL]);

        #WIND SPEED [M S-1]
        Collection_ux=ux.select(['ux_G'],[NAME_FINAL]);
        #RELATIVE HUMIDITY [%]
        Collection_rh = UR.select(['RH_G'],[NAME_FINAL]);
    else:
        Collection_ET=Collection_ET.addBands(ET_daily)
        Collection_H=Collection_H.addBands(ls_trial.select(['H'],[NAME_FINAL]))
        Collection_Rn_daily=Collection_Rn_daily.addBands(Rn24hobs.select(['Rn24h_G'],[NAME_FINAL]))
        Collection_Ta=Collection_Ta.addBands(T_air.select(['AirT_G'],[NAME_FINAL]))
        Collection_ux=Collection_ux.addBands(ux.select(['ux_G'],[NAME_FINAL]))
        Collection_rh=Collection_rh.addBands(UR.select(['RH_G'],[NAME_FINAL]))
    k=k+1
    print(k)
    n=n+1
    print(n)

['LC08_038037_20210207', 'LC08_038037_20210223', 'LC08_038037_20210327', 'LC08_038037_20210412', 'LC08_038037_20210428', 'LC08_038037_20210514', 'LC08_038037_20210530', 'LC08_038037_20210701', 'LC08_038037_20210802', 'LC08_038037_20210919', 'LC08_038037_20211021', 'LC08_038037_20211106', 'LC08_038037_20211122']
Number of scenes:  13
LC08_L1TP_038037_20210207_20210305_01_T1
1
1
LC08_L1TP_038037_20210223_20210304_01_T1
2
2
LC08_L1TP_038037_20210327_20210402_01_T1
3
3
LC08_L1TP_038037_20210412_20210423_01_T1
4
4
LC08_L1TP_038037_20210428_20210507_01_T1
5
5
LC08_L1TP_038037_20210514_20210525_01_T1
6
6
LC08_L1TP_038037_20210530_20210608_01_T1
7
7
LC08_L1TP_038037_20210701_20210708_01_T1
8
8
LC08_L1TP_038037_20210802_20210811_01_T1
9
9
LC08_L1TP_038037_20210919_20210925_01_T1
10
10
LC08_L1TP_038037_20211021_20211102_01_T1
11
11
LC08_L1TP_038037_20211106_20211116_01_T1
12
12
LC08_L1TP_038037_20211122_20211130_01_T1
13
13


In [19]:
time_start_list

['1810',
 '1810',
 '189',
 '189',
 '189',
 '189',
 '189',
 '1810',
 '1810',
 '1810',
 '1810',
 '1810',
 '1810']

In [23]:
def et_collection_SR(start_date,end_date,path,row):
    ls=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date,end_date).filterMetadata('WRS_PATH', 'equals', path).filterMetadata('WRS_ROW', 'equals',row).filterMetadata('CLOUD_COVER', 'less_than', 50);
#     ls = ls.filter(ee.Filter.neq('system:index','LC08_038037_20210903'));
    ls_list=ls.aggregate_array('system:index').getInfo()
    print(ls_list)
    count = ls.size().getInfo()
    print("Number of scenes: ", count)
    n=0
    k=0
    #====== ITERATIVE PROCESS ======#
    #FOR EACH IMAGE ON THE LIST
    #ESTIMATE ET DAILY IMAGE
    while n < count:
        #GET IMAGE
        image= ls.filterMetadata('system:index','equals',ls_list[n]).first()
        image.getInfo()
        image=ee.Image(image)
            # et=image.Image(image)
        NDVI_cold=5
        Ts_cold=20
        NDVI_hot=10
        Ts_hot=20
        index=image.get('system:index')
        cloud_cover=image.get('CLOUD_COVER')
        LANDSAT_ID=image.get('LANDSAT_ID').getInfo()
        print(LANDSAT_ID)
        landsat_version=image.get('SATELLITE').getInfo()
        azimuth_angle=image.get('SOLAR_ZENITH_ANGLE')
        time_start=image.get('system:time_start')
        date=ee.Date(time_start)
        year=ee.Number(date.get('year'))
        month=ee.Number(date.get('month'))
        day=ee.Number(date.get('day'))
        hour=ee.Number(date.get('hour'))
        minuts = ee.Number(date.get('minutes'))
        crs = image.projection().crs()
        transform=ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))
        date_string=date.format('YYYY-MM-dd').getInfo()
        #ENDMEMBERS
        p_top_NDVI=ee.Number(NDVI_cold)
        p_coldest_Ts=ee.Number(Ts_cold)
        p_lowest_NDVI=ee.Number(NDVI_hot)
        p_hottest_Ts=ee.Number(Ts_hot)
        ls_trial=image.select([0,1,2,3,4,5,6,7,10], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","BRT","pixel_qa"])
        ls.first_toa=ee.Image('LANDSAT/LC08/C01/T1/'+index.getInfo())

        col_rad = ee.Algorithms.Landsat.calibratedRadiance(ls.first_toa)
        col_rad = ls_trial.addBands(col_rad.select([9],["T_RAD"]))
        #CLOUD REMOVAL
        # ls_trial=ee.ImageCollection(col_rad).map(masks.f_cloudMaskL8_SR)
        ls_trial=masks.f_cloudMaskL8_SR(ls_trial)
        #ALBEDO TASUMI ET AL. (2008) METHOD WITH KE ET AL. (2016) COEFFICIENTS
        # ls_trial=ls_trial.map(masks.f_albedoL8)
        ls_trial=masks.f_albedoL8(ls_trial)
        #------ Meteorology
          #GEOMETRY
        geometryReducer=ls_trial.geometry().bounds().getInfo()
        geometry_download=geometryReducer['coordinates']
        # camada_clip=ls_trial.select('BRT').first()
        camada_clip=ls_trial.select('BRT')
        sun_elevation=ee.Number(90).subtract(ee.Number(azimuth_angle))
        #METEOROLOGY PARAMETERS
        col_meteorology= meteorology.get_meteorology(ls_trial,time_start);

        #AIR TEMPERATURE [C]
        T_air = col_meteorology.select('AirT_G');

        #WIND SPEED [M S-1]
        ux= col_meteorology.select('ux_G');
        #RELATIVE HUMIDITY [%]
        UR = col_meteorology.select('RH_G');

        #NET RADIATION 24H [W M-2]
        Rn24hobs = col_meteorology.select('Rn24h_G');
        #------
        #------ Elevation
        #SRTM DATA ELEVATION
        SRTM_ELEVATION ='USGS/SRTMGL1_003'
        srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
        z_alt = srtm.select('elevation')

    #     GET IMAGE
    #     ls_trial=ls_trial.first()

    #     SPECTRAL IMAGES (NDVI, EVI, SAVI, LAI, T_LST, e_0, e_NB, long, lat)
        ls_trial=tools.fexp_spec_ind(ls_trial)
        ls_trial=tools.LST_DEM_correction(ls_trial, z_alt, T_air, UR,sun_elevation,hour,minuts)
        #COLD PIXEL
        d_cold_pixel=endmembers.fexp_cold_pixel(ls_trial, geometryReducer, p_top_NDVI, p_coldest_Ts)
        #COLD PIXEL NUMBER
        n_Ts_cold = ee.Number(d_cold_pixel.get('temp').getInfo())
        #INSTANTANEOUS OUTGOING LONG-WAVE RADIATION [W M-2]
        ls_trial=tools.fexp_radlong_up(ls_trial)

        #INSTANTANEOUS INCOMING SHORT-WAVE RADIATION [W M-2]
        ls_trial=tools.fexp_radshort_down(ls_trial,z_alt,T_air,UR, sun_elevation)

        #INSTANTANEOUS INCOMING LONGWAVE RADIATION [W M-2]
        ls_trial=tools.fexp_radlong_down(ls_trial, n_Ts_cold)
        #INSTANTANEOUS NET RADIATON BALANCE [W M-2]
    #     print(ls_trial.select('Rs_down').getInfo())
    #     print(ls_trial.select('Rl_down').getInfo())
    #     print(ls_trial.select('Rl_up').getInfo())

        ls_trial=tools.fexp_radbalance(ls_trial)
    #     print(ls_trial.select('Rn').getInfo())

        #SOIL HEAT FLUX (G) [W M-2]
        ls_trial=tools.fexp_soil_heat(ls_trial)
        #HOT PIXEL
        d_hot_pixel=endmembers.fexp_hot_pixel(ls_trial, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
        #SENSIBLE HEAT FLUX (H) [W M-2]
        ls_trial=tools.fexp_sensible_heat_flux(ls_trial, ux, UR,Rn24hobs,n_Ts_cold,
                                       d_hot_pixel, date_string,geometryReducer)

        #DAILY EVAPOTRANSPIRATION (ET_24H) [MM DAY-1]
        ls_trial=evapotranspiration.fexp_et(ls_trial,Rn24hobs)

        NAME_FINAL=LANDSAT_ID[:5]+LANDSAT_ID[10:17]+LANDSAT_ID[17:25]
        ls_trial=ls_trial.addBands([ls_trial.select('ET_24h').rename(NAME_FINAL)])
        ET_daily=ls_trial.select(['ET_24h'],[NAME_FINAL])
        if k ==0:
            Collection_ET=ET_daily
            Collection_H=ls_trial.select(['H'],[NAME_FINAL])
            Collection_Rn_daily=Rn24hobs.select(['Rn24h_G'],[NAME_FINAL])
            Collection_ndvi=ls_trial.select(["NDVI"],[NAME_FINAL])
              #AIR TEMPERATURE [C]
            Collection_Ta=T_air.select(['AirT_G'],[NAME_FINAL]);

            #WIND SPEED [M S-1]
            Collection_ux=ux.select(['ux_G'],[NAME_FINAL]);
            #RELATIVE HUMIDITY [%]
            Collection_rh = UR.select(['RH_G'],[NAME_FINAL]);

        else:
            Collection_ET=Collection_ET.addBands(ET_daily)
            Collection_H=Collection_H.addBands(ls_trial.select(['H'],[NAME_FINAL]))
            Collection_Rn_daily=Collection_Rn_daily.addBands(Rn24hobs.select(['Rn24h_G'],[NAME_FINAL]))
            Collection_Ta=Collection_Ta.addBands(T_air.select(['AirT_G'],[NAME_FINAL]))
            Collection_ux=Collection_ux.addBands(ux.select(['ux_G'],[NAME_FINAL]))
            Collection_rh=Collection_rh.addBands(UR.select(['RH_G'],[NAME_FINAL]))
            Collection_ndvi=Collection_ndvi.addBands(ls_trial.select(['NDVI'],[NAME_FINAL]))
       
        k=k+1
        print(k)
        n=n+1
        print(n)
    return(Collection_ET,Collection_H,Collection_Rn_daily,Collection_ndvi)
ET,H,Rn_daily,ndvi=et_collection_SR("2021-04-01","2021-09-30",30,34)
ET=ET.reproject(crs="EPSG:4326",scale=30)
H=H.reproject(crs="EPSG:4326",scale=30)
Rn_daily=Rn_daily.reproject(crs="EPSG:4326",scale=30)
ndvi=ndvi.reproject(crs="EPSG:4326",scale=30)

['LC08_030034_20210404', 'LC08_030034_20210420', 'LC08_030034_20210506', 'LC08_030034_20210607', 'LC08_030034_20210623', 'LC08_030034_20210709', 'LC08_030034_20210810', 'LC08_030034_20210826', 'LC08_030034_20210911', 'LC08_030034_20210927']
Number of scenes:  10
LC08_L1TP_030034_20210404_20210409_01_T1
1
1
LC08_L1TP_030034_20210420_20210430_01_T1
2
2
LC08_L1TP_030034_20210506_20210517_01_T1
3
3
LC08_L1TP_030034_20210607_20210615_01_T1
4
4
LC08_L1TP_030034_20210623_20210629_01_T1
5
5
LC08_L1TP_030034_20210709_20210720_01_T1
6
6
LC08_L1TP_030034_20210810_20210819_01_T1
7
7
LC08_L1TP_030034_20210826_20210901_01_T1
8
8
LC08_L1TP_030034_20210911_20210916_01_T1
9
9
LC08_L1TP_030034_20210927_20211001_01_T1
10
10


In [15]:

## * Applicable only if the features lie in one scene
import pandas as pd
features=[ee.Geometry.Point([ -114.736906,33.439129 ]),
         ee.Geometry.Point([-114.715314,33.547044 ])]
names=["South","North"]
def extract_pixel_vals(ls_collection,features,var_name,point_names):
    ls_collection=ls_collection.reproject(crs="EPSG:4326",scale=30)
    point_df=[]
    for i in range(len(features)):
        var=ls_collection.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= features[i],
        scale= 30,
        maxPixels=9000000000)
        [print(j) for j in list(var.getInfo())]
        dates=pd.to_datetime([j.split("_")[2] for j in list(var.getInfo())])
        vals=[item for item in var.getInfo().values()]
        temp=pd.DataFrame({str(var_name)+"_"+point_names[i]:vals},index=dates)
        point_df.append(temp)
    return(pd.concat(point_df,axis=1))
extract_pixel_vals(Collection_ET,features,"ET_GEE_mm",names)

NameError: name 'Collection_ET' is not defined

In [19]:
# 36.969566272325714, -102.20540138129319
## Automating pixel extraction of multiple sites in one image
## * Applicable only if the features lie in one scene
import pandas as pd
features=[ee.Geometry.Point([  -99.897984,36.894757])]
names=["ok"]
def extract_pixel_vals(ls_collection,features,var_name,point_names):
    ls_collection=ls_collection.reproject(crs="EPSG:4326",scale=30)
    point_df=[]
    for i in range(len(features)):
        var=ls_collection.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= features[i],
        scale= 30,
        maxPixels=9000000000)
        [print(j) for j in list(var.getInfo())]
        dates=pd.to_datetime([j.split("_")[2] for j in list(var.getInfo())])
        vals=[item for item in var.getInfo().values()]
        temp=pd.DataFrame({str(var_name)+"_"+point_names[i]:vals},index=dates)
        point_df.append(temp)
    return(pd.concat(point_df,axis=1))
# extract_pixel_vals(Collection_ET,features,"ET_GEE_mm",names)

In [20]:
#Merge the different variables
band_names=[ET,H,Rn_daily,ndvi]
var_names=["ET_GEE_mm","H_GEE","Rn_daily_GEE","ndvi"]
def merged_df_landsat(band_names,features,var_names,names):
    var=[]
    for i in range(len(band_names)):
        var.append(extract_pixel_vals(band_names[i],features,var_names[i],names))
    return(pd.concat(var,axis=1))
sebal_vars_pv=merged_df_landsat(band_names,features,var_names,names) 
sebal_vars_pv


LC08_030034_20210404
LC08_030034_20210420
LC08_030034_20210506
LC08_030034_20210607
LC08_030034_20210623
LC08_030034_20210709
LC08_030034_20210810
LC08_030034_20210826
LC08_030034_20210911
LC08_030034_20210927
LC08_030034_20210404
LC08_030034_20210420
LC08_030034_20210506
LC08_030034_20210607
LC08_030034_20210623
LC08_030034_20210709
LC08_030034_20210810
LC08_030034_20210826
LC08_030034_20210911
LC08_030034_20210927
LC08_030034_20210404
LC08_030034_20210420
LC08_030034_20210506
LC08_030034_20210607
LC08_030034_20210623
LC08_030034_20210709
LC08_030034_20210810
LC08_030034_20210826
LC08_030034_20210911
LC08_030034_20210927


,ET_GEE_mm_ok,H_GEE_ok,Rn_daily_GEE_ok
2021-04-04,3.908222,139.419407,155.250748
2021-04-20,2.628224,278.354372,140.123364
2021-05-06,5.398813,147.269347,201.428461
2021-06-07,NaN,NaN,NaN
2021-06-23,6.533777,22.206562,191.259267
2021-07-09,5.331600,131.066941,200.064375
2021-08-10,3.775493,171.675279,168.295043
2021-08-26,5.536422,49.691177,170.693001
2021-09-11,3.554070,153.178448,147.079774
2021-09-27,3.523652,21.686664,103.498902


In [7]:
#Merge the different variables
band_names=[Collection_ET,Collection_H,Collection_Rn_daily,Collection_Ta,Collection_ux,Collection_rh]
var_names=["ET_GEE_mm","H_GEE","Rn_daily_GEE","Ta_ERA","u_x_ERA","RH_ERA"]
def merged_df_landsat(band_names,features,var_names,names):
    var=[]
    for i in range(len(band_names)):
        var.append(extract_pixel_vals(band_names[i],features,var_names[i],names))
    return(pd.concat(var,axis=1))
sebal_vars_pv=merged_df_landsat(band_names,features,var_names,names) 
sebal_vars_pv


NameError: name 'Collection_ET' is not defined

In [10]:
sebal_vars_pv.to_csv("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\ET_direct_data\\GEE_SEBAL\\GEE_SEB_PVID.csv")

In [6]:
#------- Extract time series 
st1 =  ee.Geometry.Point([ -114.736906,33.439129 ])
st2=ee.Geometry.Point([-114.715314,33.547044 ])
st1.getInfo()
et1 = Collection_ET.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= st1,
        scale= 30,
        maxPixels=9000000000)
et2 = Collection_ET.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= st2,
        scale= 30,
        maxPixels=9000000000)

Rn_daily1= Collection_Rn_daily.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= st1,
        scale= 30,
        maxPixels=9000000000)
Rn_daily2= Collection_Rn_daily.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= st2,
        scale= 30,
        maxPixels=9000000000)
Ta_1= Collection_Ta.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= st1,
        scale= 30,
        maxPixels=9000000000)
Ta2= Collection_Ta.reduceRegion(
        reducer= ee.Reducer.first(),
        geometry= st2,
        scale= 30,
        maxPixels=9000000000)

In [ ]:
et1_dict=et1.getInfo()
et2_dict=et2.getInfo()
Rnd1_dict_dict=Rn_daily1.getInfo()
Rnd2_dict=Rn_daily2.getInfo()

In [ ]:
dict_keys=list(et1_dict.keys())
date=[]
for i in dict_keys:
    date.append(i.split("_")[2])

In [ ]:
import pandas as pd 
date_pd=pd.Series(date)
dates_2021=pd.to_datetime(date_pd,format='%Y%m%d')
etst1=pd.Series(list(et1_dict.values()))
etst2=pd.Series(list(et2_dict.values()))
rnst1=pd.Series(list(Rnd1_dict_dict.values()))
rnst2=pd.Series(list(Rnd2_dict.values()))

In [ ]:
landsat_2021=pd.DataFrame()
landsat_2021["date"]=dates_2021
landsat_2021["et_st1"]=etst1
landsat_2021["Rnst1"]=rnst1
landsat_2021["et_st2"]=etst2
landsat_2021["Rnst2"]=rnst2

In [ ]:
landsat_2021.to_csv("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\ET_results\\csv_files\\landsat_2021_EC(2).csv")

In [ ]:
# --------- Plotting 
import matplotlib as pyplot 
import matplotlib.dates as mdates
fig,ax=plt.subplots(figsize=(10,7))
date_format = mdates.DateFormatter('%d-%m')
plt.gca().xaxis.set_major_formatter(date_format)
ax.plot(landsat_2021["date"],landsat_2021["et_st1"],"b",label="ET(W/m2)")
ax.plot(landsat_2021["date"],landsat_2021["et_st1"],'o',c="b")

ax.plot(landsat_2021["date"],landsat_2021["Rnst1"],"r",label="Net Radiation(W/m2)")
ax.plot(landsat_2021["date"],landsat_2021["Rnst1"],"o",c="r")
ax.set_title("EC Station 1",fontsize=20)
ax.set_ylabel("W/m2",fontsize=20)
# ax.set_yticks(np.linspace(20,200,10))
ax.set_yticklabels(np.linspace(20,200,10),fontsize=15)
ax.set_ylim(20,220)
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.grid()
# plt.savefig("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\figures\\EC_Station1_Timeseries2021.png",dpi=300,bbox="tight")

In [ ]:
# --------- Plotting 
import matplotlib as pyplot 
import matplotlib.dates as mdates
fig,ax=plt.subplots(figsize=(10,7))
date_format = mdates.DateFormatter('%m-%y')
plt.gca().xaxis.set_major_formatter(date_format)
ax.plot(landsat_2021["date"],landsat_2021["et_st1"],"b")
ax.plot(landsat_2021["date"],landsat_2021["et_st1"],'o',c="b")

# ax.plot(landsat_2021["date"],landsat_2021["Rnst1"],"r",label="Net Radiation(W/m2)")
# ax.plot(landsat_2021["date"],landsat_2021["Rnst1"],"o",c="r")
ax.set_title("EC Station 1",fontsize=20)
ax.set_ylabel("ET(mm)",fontsize=20)
# ax.set_yticks(np.linspace(20,200,10))
# ax.set_yticklabels(np.linspace(20,200,10),fontsize=15)
# ax.set_ylim(20,220)
plt.legend(fontsize=15)
plt.xticks(fontsize=12)

plt.yticks(fontsize=15)
plt.grid()
plt.savefig("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\figures\\EC_Station1_Timeseries2021_mm.png",dpi=300,bbox="tight")

In [1]:
#--------- Plotting 
import matplotlib as pyplot 
import matplotlib.dates as mdates
fig,ax=plt.subplots(figsize=(10,7))
date_format = mdates.DateFormatter('%m-%y')
plt.gca().xaxis.set_major_formatter(date_format)
ax.plot(landsat_2021["date"],landsat_2021["et_st2"],"b",label="ET(W/m2)")
ax.plot(landsat_2021["date"],landsat_2021["et_st2"],'o',c="b")
# ax.plot(ecostress2["et_ecost2_date"],ecostress2["et_ecost2"],c="k")
ax.plot(landsat_2021["date"],landsat_2021["Rnst2"],"r",label="Net Radiation(W/m2)")
ax.plot(landsat_2021["date"],landsat_2021["Rnst2"],"o",c="r")
ax.set_title("EC Station 2",fontsize=20)
ax.set_ylabel("W/m2",fontsize=20)
ax.set_yticklabels(np.linspace(20,200,10),fontsize=15)
ax.set_ylim(20,220)
plt.xticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()
# plt.savefig("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\figures\\EC_Station2_Timeseries2021.png",dpi=300,bbox="tight")

NameError: name 'plt' is not defined

In [ ]:
#--------- Plotting 
import matplotlib as pyplot 
import matplotlib.dates as mdates
fig,ax=plt.subplots(figsize=(10,7))
date_format = mdates.DateFormatter('%m-%y')
plt.gca().xaxis.set_major_formatter(date_format)
ax.plot(landsat_2021["date"],landsat_2021["et_st2"],"b")
ax.plot(landsat_2021["date"],landsat_2021["et_st2"],'o',c="b")
# ax.plot(ecostress2["et_ecost2_date"],ecostress2["et_ecost2"],c="k")
# ax.plot(landsat_2021["date"],landsat_2021["Rnst2"],"r",label="Net Radiation(W/m2)")
# ax.plot(landsat_2021["date"],landsat_2021["Rnst2"],"o",c="r")
ax.set_title("EC Station 2",fontsize=20)
ax.set_ylabel("ET(mm)",fontsize=20)
# ax.set_yticklabels(np.linspace(20,200,10),fontsize=15)
# ax.set_ylim(20,220)
plt.xticks(fontsize=12)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()
# plt.savefig("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\figures\\EC_Station2_Timeseries2021_mm.png",dpi=300,bbox="tight")

In [ ]:
#Read ECOSTRESS point data 
import os
os.chdir("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\ET_results\\csv_files\\ec-palo-verde\\")
eco=pd.read_csv("EC-Palo-verde-ECO3ETPTJPL-001-results.csv")
eco["date"]=eco["Date"].str.split().str[0]
eco["date"]=pd.to_datetime(eco["date"])
print(eco["Latitude"].unique())
eco.columns

In [ ]:
ecostress1=pd.DataFrame()
ecostress2=pd.DataFrame()
ecostress1["et_ecost1"]=eco[eco["Latitude"]==eco["Latitude"].unique()[0]]['ECO3ETPTJPL_001_EVAPOTRANSPIRATION_PT_JPL_ETdaily']
ecostress2["et_ecost2"]=eco[eco["Latitude"]==eco["Latitude"].unique()[1]]['ECO3ETPTJPL_001_EVAPOTRANSPIRATION_PT_JPL_ETdaily']
ecostress1["et_ecost1_date"]=eco[eco["Latitude"]==eco["Latitude"].unique()[0]]['date']
ecostress2["et_ecost2_date"]=eco[eco["Latitude"]==eco["Latitude"].unique()[1]]['date']
ecostress2

In [ ]:
Collection_ET.select('LC08_038037_20210207').getInfo()## Create function for plotting layers 
def plot_scenes(image,bands,minm,maxm,palette,name):
    fig = plt.figure(figsize=(15, 10))

    # here is the bounding box of the map extent we want to use
    # formatted a [E,S,W,N]
#     zoom_region = [-112.79, 32.049037,-115.368771, 34.32]
    vis_params = {"bands":bands,
      'min': minm,
      'max': maxm,
      'palette':palette}
    # plot the map over the region of interest
    ax = cartoee.get_map(image, vis_params=vis_params)

    # add the gridlines and specify that the xtick labels be rotated 45 degrees
    # add some styling to make our map publication ready

    clb=cartoee.add_colorbar(ax,loc='right',vis_params=vis_params,orientation="horizontal",posOpts=[0.25, 0.05, 0.5, 0.02])
    cartoee.add_gridlines(ax, interval=0.5, xtick_rotation=0, linestyle="--")
    ax.grid(False)
    # add coastline
    ax.coastlines(color="cyan")
    ax.set_title(label = name, fontsize=20)
#     plt.savefig("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\figures\\"+name+".png",bbox="tight",dpi=300)
# plot_scenes(ls_trial,["ALFA"],0,0.4,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"albedo")
# plot_scenes(SW,["SW_Down"],290,300,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"SW_daily")
# plot_scenes(Tair,['AirT_G'],15,25,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Tair")
# plot_scenes(Rn24hobs,['Rn24h_G'],90,200,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rndaily")
# plot_scenes(ls_trial,['NDVI'],-0.1,0.8,cm.get_palette(cmap_name='RdYlGn', n_class=8, hashtag=False),"NDVI")
# plot_scenes(ls_trial,['SAVI'],-0.1,0.8,cm.get_palette(cmap_name='RdYlGn', n_class=8, hashtag=False),"SAVI")
# plot_scenes(ls_trial,['LAI'],0,3,cm.get_palette(cmap_name='RdYlGn', n_class=8, hashtag=False),"LAI")
# plot_scenes(ls_trial,['e_0'],0.95,0.953,cm.get_palette(cmap_name='RdYlBu', n_class=8, hashtag=False),"Broadband_e")
# plot_scenes(ls_trial,['NDWI'],-0.8,0.5,cm.get_palette(cmap_name='RdYlBu', n_class=8, hashtag=False),"NDWI")
# plot_scenes(ls_trial,['T_LST_DEM'],290,320,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"LST_corrected")
# plot_scenes(ls_trial,['T_LST'],290,320,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"LST")
# plot_scenes(ls_trial,['Rs_down'],800,900,cm.get_palette(cmap_name='YlOrRd', n_class=8, hashtag=False),"Rs_inst")
# plot_scenes(ls_trial,['Rl_up'],400,550,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rl_out")
# plot_scenes(ls_trial,['Rl_down'],330,335,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rl_in")
# plot_scenes(ls_trial,['Rn'],400,700,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rn_inst")
# plot_scenes(ls_trial,['G'],0,150,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"G")
# plot_scenes(Collection_ET,['LC08_038037_20210207'],0,5,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"elevation")

plot_scenes(i_Rn_24h ,['Rn24h_G'],0,500,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"elevation")



In [ ]:
Collection_ET.select('LC08_038037_20210207').getInfo()## Create function for plotting layers 
def plot_scenes(image,bands,minm,maxm,palette,name):
    fig = plt.figure(figsize=(15, 10))

    # here is the bounding box of the map extent we want to use
    # formatted a [E,S,W,N]
#     zoom_region = [-112.79, 32.049037,-115.368771, 34.32]
    vis_params = {"bands":bands,
      'min': minm,
      'max': maxm,
      'palette':palette}
    # plot the map over the region of interest
    ax = cartoee.get_map(image, vis_params=vis_params)

    # add the gridlines and specify that the xtick labels be rotated 45 degrees
    # add some styling to make our map publication ready

    clb=cartoee.add_colorbar(ax,loc='right',vis_params=vis_params,orientation="horizontal",posOpts=[0.25, 0.05, 0.5, 0.02])
    cartoee.add_gridlines(ax, interval=0.5, xtick_rotation=0, linestyle="--")
    ax.grid(False)
    # add coastline
    ax.coastlines(color="cyan")
    ax.set_title(label = name, fontsize=20)
#  plt.savefig("C:\\Rouhin_Lenovo\\US_project\\Alfalfa\\figures\\"+name+".png",bbox="tight",dpi=300)
# plot_scenes(ls_trial,["ALFA"],0,0.4,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"albedo")
# plot_scenes(SW,["SW_Down"],290,300,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"SW_daily")
# plot_scenes(Tair,['AirT_G'],15,25,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Tair")
# plot_scenes(Rn24hobs,['Rn24h_G'],90,200,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rndaily")
# plot_scenes(ls_trial,['NDVI'],-0.1,0.8,cm.get_palette(cmap_name='RdYlGn', n_class=8, hashtag=False),"NDVI")
# plot_scenes(ls_trial,['SAVI'],-0.1,0.8,cm.get_palette(cmap_name='RdYlGn', n_class=8, hashtag=False),"SAVI")
# plot_scenes(ls_trial,['LAI'],0,3,cm.get_palette(cmap_name='RdYlGn', n_class=8, hashtag=False),"LAI")
# plot_scenes(ls_trial,['e_0'],0.95,0.953,cm.get_palette(cmap_name='RdYlBu', n_class=8, hashtag=False),"Broadband_e")
# plot_scenes(ls_trial,['NDWI'],-0.8,0.5,cm.get_palette(cmap_name='RdYlBu', n_class=8, hashtag=False),"NDWI")
# plot_scenes(ls_trial,['T_LST_DEM'],290,320,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"LST_corrected")
# plot_scenes(ls_trial,['T_LST'],290,320,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"LST")
# plot_scenes(ls_trial,['Rs_down'],800,900,cm.get_palette(cmap_name='YlOrRd', n_class=8, hashtag=False),"Rs_inst")
# plot_scenes(ls_trial,['Rl_up'],400,550,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rl_out")
# plot_scenes(ls_trial,['Rl_down'],330,335,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rl_in")
# plot_scenes(ls_trial,['Rn'],400,700,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"Rn_inst")
# plot_scenes(ls_trial,['G'],0,150,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"G")
plot_scenes(Collection_ET,['LC08_038037_20210223'],0,5,cm.get_palette(cmap_name='RdYlBu_r', n_class=8, hashtag=False),"elevation")